In [4]:
library(tidyverse)
library(ggpubr)
library(yaml)
library(patchwork)

#read config
config <- read_yaml(file = "../HiRES_preprocess_pipeline//config.yaml")

#Common
Raw <- read_table2("../stat/raw.fq.stat",col_names=F) %>% arrange(X1) %>% rowwise() %>% mutate(X2 = X2/4*300/1000000000,X1 = strsplit(X1,split = "/")[[1]][3],X1 = strsplit(X1,split = "_")[[1]][1])
DNA <- read_table2("../stat/dna.fq.stat",col_names=F) %>% arrange(X1) %>% rowwise()%>% mutate(X2 = X2/4*300/1000000000,X1 = strsplit(X1,split = "/")[[1]][3])
RNA <- read_table2("../stat/rna.fq.stat",col_names=F) %>% arrange(X1) %>% rowwise() %>% mutate(X2 = X2/4*300/1000000000,X1 = strsplit(X1,split = "/")[[1]][3])

raw_pairs <- read_table2("../stat/raw.pairs.stat",col_names=F) %>% arrange(X1) %>% rowwise() %>% mutate(X1 = strsplit(X1,split = "/")[[1]][3])
pairs_dedup <- read_table2("../stat/pairs.dedup.stat",col_names=F) %>% arrange(X1) %>% rowwise() %>% mutate(X1 = strsplit(X1,split = "/")[[1]][3])

pairs_c1 <- read_table2("../stat/pairs.c1.stat",col_names=F)%>% arrange(X1) %>%rowwise() %>% mutate(X1 = strsplit(X1,split = "/")[[1]][5]) %>% mutate(X1 = str_replace(X1,".pairs.gz",""))
pairs_c12 <- read_table2("../stat/pairs.c12.stat",col_names=F)%>% arrange(X1) %>%rowwise() %>% mutate(X1 = strsplit(X1,split = "/")[[1]][5]) %>% mutate(X1 = str_replace(X1,".pairs.gz",""))
pairs_c123 <- read_table2("../stat/pairs.c123.stat",col_names=F)%>% arrange(X1) %>%rowwise() %>% mutate(X1 = strsplit(X1,split = "/")[[1]][5]) %>% mutate(X1 = str_replace(X1,".pairs.gz",""))
inter_pairs_c123 <- read_table2(file = "../stat/inter.pairs.c123.stat",col_names = F) %>% arrange(X1) %>%rowwise() %>% mutate(X1 = strsplit(X1,split = "/")[[1]][5]) %>% mutate(X1 = str_replace(X1,".pairs.gz",""))

yperx <- read_table2("../stat/yperx.stat",col_names=F) %>% arrange(X1) %>% rowwise() %>% mutate(X1 = strsplit(X1,split = "/")[[1]][2])
stat <- Raw %>% left_join(DNA,by="X1") %>% left_join(RNA,by="X1") %>% left_join(yperx,by="X1") %>% left_join(raw_pairs,by="X1") %>% left_join(pairs_dedup,by="X1") %>% 
    left_join(pairs_c1,by="X1") %>% left_join(pairs_c12,by="X1")%>% left_join(pairs_c123,by="X1") %>% left_join(inter_pairs_c123,by="X1")
RNAres <- read_table2("../result/RNA_Res/counts.gene.total.format.tsv")
featureStat <- cbind(as.data.frame(colSums(RNAres %>% select(-gene))),colSums(RNAres %>% select(-gene) != 0)) %>% rownames_to_column("X1")
RNAresExon <- read_table2("../result/RNA_Res/counts.exon.total.format.tsv")
featureStatExon <- cbind(as.data.frame(colSums(RNAresExon %>% select(-gene))),colSums(RNAresExon %>% select(-gene) != 0))  %>% rownames_to_column("X1")

if(config$if_RNA_snp_split){
    RNAresGenome1 <- read_table2("../result/RNA_Res/counts.gene.genome1.tsv")
    RNAresGenome2 <- read_table2("../result/RNA_Res/counts.gene.genome2.tsv")
    featureStatGenome1 <- cbind(as.data.frame(colSums(RNAresGenome1 %>% select(-gene))),colSums(RNAresGenome1 %>% select(-gene) != 0)) %>% rownames_to_column("X1")
    featureStatGenome2 <- cbind(as.data.frame(colSums(RNAresGenome2 %>% select(-gene))),colSums(RNAresGenome2 %>% select(-gene) != 0)) %>% rownames_to_column("X1")
    stat <- stat %>% left_join(featureStat,by="X1") %>% left_join(featureStatExon,by="X1") %>% left_join(featureStatGenome1,by="X1")  %>% left_join(featureStatGenome2,by="X1")
    names(stat) <- c("cellname","Rawreads","DNAreads","RNAreads","yperx","raw_pairs","pairs_dedup","pairs_clean1","pairs_clean2","pairs_clean3","inter_pairs_clean3",
                     "UMIs_gene","genes_gene","UMIs_exon","genes_exon","UMIs_gene_genome1","genes_gene_genome1","UMIs_gene_genome2","genes_gene_genome2")
} else{
    stat <- stat %>% left_join(featureStat,by="X1") %>% left_join(featureStatExon,by="X1") 
    names(stat) <- c("cellname","Rawreads","DNAreads","RNAreads","yperx","raw_pairs","pairs_dedup","pairs_clean1","pairs_clean2","pairs_clean3","inter_pairs_clean3",
                     "UMIs_gene","genes_gene","UMIs_exon","genes_exon")
}


if(config$if_structure){
    rmsd <- read_table2("../stat/rmsd.info",col_names = F) %>% rowwise() %>% mutate(X1=str_split(X1,pattern = fixed("/")),cellname = X1[1], res = str_split(X1[3], fixed("."))[[1]][2],TOP3_RMSD = X5) %>% select(cellname,res,TOP3_RMSD) %>% spread(res,TOP3_RMSD) %>% arrange(cellname)
    stat <- stat %>% left_join(rmsd)
}




Warning message:
“`read_table2()` was deprecated in readr 2.0.0.
ℹ Please use `read_table()` instead.”

── Column specification ──────────────────────────────────────────────────────────────────────────────────────────────────────────────
cols(
  X1 = col_character(),
  X2 = col_double()
)


── Column specification ──────────────────────────────────────────────────────────────────────────────────────────────────────────────
cols(
  X1 = col_character(),
  X2 = col_double()
)


── Column specification ──────────────────────────────────────────────────────────────────────────────────────────────────────────────
cols(
  X1 = col_character(),
  X2 = col_double()
)


── Column specification ──────────────────────────────────────────────────────────────────────────────────────────────────────────────
cols(
  X1 = col_character(),
  X2 = col_double()
)


── Column specification ──────────────────────────────────────────────────────────────────────────────────────────────────────────────
cols(

In [5]:
plot <- stat %>% mutate(RNAreadsRatio = RNAreads/(RNAreads+DNAreads),
                        pairsPerRead = raw_pairs/DNAreads/1000000000*300,
                        pairsValidRatio = pairs_clean3/raw_pairs,
                        interPairsRatio = inter_pairs_clean3/pairs_clean3) 
#plot[is.na(plot)]<-0

In [6]:
options(repr.matrix.max.cols=50, repr.matrix.max.rows=100)
plot %>% summary()

   cellname            Rawreads        DNAreads        RNAreads      
 Length:8           Min.   :1.460   Min.   :1.362   Min.   :0.04466  
 Class :character   1st Qu.:1.562   1st Qu.:1.497   1st Qu.:0.05291  
 Mode  :character   Median :1.595   Median :1.530   Median :0.06500  
                    Mean   :1.754   Mean   :1.678   Mean   :0.06673  
                    3rd Qu.:1.863   3rd Qu.:1.768   3rd Qu.:0.08283  
                    Max.   :2.375   Max.   :2.321   Max.   :0.08855  
     yperx            raw_pairs       pairs_dedup      pairs_clean1   
 Min.   :0.003904   Min.   :401700   Min.   :228262   Min.   :227482  
 1st Qu.:0.005102   1st Qu.:512131   1st Qu.:303439   1st Qu.:302329  
 Median :0.006709   Median :553500   Median :319448   Median :318379  
 Mean   :0.006436   Mean   :596459   Mean   :317702   Mean   :316567  
 3rd Qu.:0.007266   3rd Qu.:641453   3rd Qu.:347084   3rd Qu.:345956  
 Max.   :0.008881   Max.   :908234   Max.   :373964   Max.   :372480  
  pairs_clean

In [7]:
plot

cellname,Rawreads,DNAreads,RNAreads,yperx,raw_pairs,pairs_dedup,pairs_clean1,pairs_clean2,pairs_clean3,inter_pairs_clean3,UMIs_gene,genes_gene,UMIs_exon,genes_exon,1m,200k,20k,50k,RNAreadsRatio,pairsPerRead,pairsValidRatio,interPairsRatio
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
d0A1,1.576057,1.514369,0.0537867,0.006962,540415,317401,316417,282564,282357,25645,12656,4215,3540,1980,2.90887344,5.7559506,14.930391,10.2937158,0.03429933,0.10705743,0.5224818,0.09082474
d0A2,2.375023,2.320728,0.0446604,0.005020,908234,355682,354099,316409,316165,30778,6452,2661,1578,1019,2.47877086,5.1158424,14.054752,9.4978146,0.01888079,0.11740721,0.3481096,0.09734790
d0A3,1.459884,1.361508,0.0885501,0.008176,401700,228262,227482,201695,201574,10938,22466,5749,5678,2790,6.07242813,11.5597753,27.605091,19.8587936,0.06106657,0.08851213,0.5018023,0.05426295
d0A4,1.518681,1.443085,0.0666390,0.005129,529911,306581,305421,274095,273876,25036,13604,4440,3712,2099,0.07975918,0.3669357,1.578412,0.9252357,0.04413987,0.11016215,0.5168340,0.09141363
d0A5,2.145600,2.051433,0.0823998,0.003904,766147,373964,372480,336027,335813,19145,13030,4451,3499,1947,2.23562930,4.4137984,13.916148,8.7590596,0.03861587,0.11204075,0.4383141,0.05701090
d0A6,1.769414,1.673963,0.0841368,0.008881,566584,321495,320341,287257,287002,21773,15954,4763,4336,2292,3.82778516,7.9221626,20.585638,14.2469289,0.04785667,0.10154061,0.5065480,0.07586358
d0A7,1.600618,1.543057,0.0502968,0.006489,458792,294012,293052,266166,265991,21905,12729,4305,3268,1915,1.81597914,3.5121959,8.750729,6.1813970,0.03156663,0.08919801,0.5797638,0.08235241
d0A8,1.588527,1.516365,0.0633612,0.006928,599888,344218,343241,313776,313565,30567,12226,4407,3368,1951,0.90000459,1.7111580,4.595248,3.0936953,0.04010897,0.11868275,0.5227059,0.09748218
